<a href="https://colab.research.google.com/github/michaelfarayola7/Data-Science-ML-Projects/blob/main/Building_a_Virtual_Keyboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using/Displaying your camera

#### Libraries to Install
* cv2
* cvzone
* mediapipe
* pyautogui

In [ ]:
import cv2

# Open the default camera
cap = cv2.VideoCapture(0)

while True:
    # Read a new frame from the camera
    success, img = cap.read()

    # Check if the frame was successfully read
    if not success:
        break

    # Display the frame in a window named 'camera'
    cv2.imshow('camera', img)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()



In [ ]:
import cv2
import mediapipe as mp
from time import sleep
import pyautogui as keyboard

# Open the default camera
cap = cv2.VideoCapture(0)

# Set the dimensions of the camera frame
cap.set(3, 1280)
cap.set(4, 720)

keys = [["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]

class Button():
    def __init__(self, pos, text, size=[80,80]):
        self.pos = pos
        self.text = text
        self.size = size
        self.color = (0, 0, 0)  # Initial color

    def draw(self, img):
        x, y = self.pos
        w, h = self.size

        # Draw the rectangle and text on the image with current color
        cv2.rectangle(img, self.pos, (x+w, y+h), self.color, cv2.FILLED)
        cv2.putText(img, self.text, (x+20, y+65), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
        return img

    def set_color(self, color):
        self.color = color

# Initialize mediapipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

buttonList = []

for i in range(len(keys)):
    for j, key in enumerate(keys[i]):
        buttonList.append(Button([100 * j+50, 100 * i+100], key))


while True:
    # Read a new frame from the camera
    success, img = cap.read()

    # Check if the frame was successfully read
    if not success:
        break

    # Convert the image to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect hands in the image
    results = hands.process(img_rgb)

    # Draw landmarks and connections on the image if hands are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmark points
            for lm in hand_landmarks.landmark:
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            # Draw connections between landmarks
            connections = mp_hands.HAND_CONNECTIONS
            for connection in connections:
                point1 = connection[0]
                point2 = connection[1]
                x1, y1 = int(hand_landmarks.landmark[point1].x * w), int(hand_landmarks.landmark[point1].y * h)
                x2, y2 = int(hand_landmarks.landmark[point2].x * w), int(hand_landmarks.landmark[point2].y * h)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 2)


    # Simulate key presses when hand hovers over buttons
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                for button in buttonList:
                    x, y = button.pos
                    w, h = button.size
                    if x < cx < x+w and y < cy < y+h:
                        cv2.rectangle(img, button.pos, (x+w, y+h), (255, 0, 0), cv2.FILLED)
                        cv2.putText(img, button.text, (x+20, y+65), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
                        distance = ((lm.x - (x + w / 2)) ** 2 + (lm.y - (y + h / 2)) ** 2) ** 0.5
                        if distance < w / 2:
                            button.set_color((0, 255, 0))  # Change color to green
                            keyboard.press(button.text)
                            sleep(0.2)
                        else:
                            button.set_color((0, 0, 0))  # Change color back to black

    # Draw all buttons on the image
    for button in buttonList:
        img = button.draw(img)

    # Display the frame in a window named 'camera'
    cv2.imshow('camera', img)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from time import sleep
from pynput.keyboard import Controller

cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
detector = HandDetector(detectionCon=0.8)

keys = [["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]

ClickedText = ""
keyboard = Controller()
def drawALL(img,buttonList):

    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        cv2.rectangle(img, button.pos, (x + w, y + h), (255, 0, 255), cv2.FILLED)
        cv2.putText(img, button.text, (x + 20, y + 65), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
    return img


class Button():
    def __init__(self,pos,text,size=[80,80]):
        self.pos = pos
        self.text = text
        self.size = size


# myButton = Button([100,100],'Q')
# myButton1 = Button([200,100],'W')
# myButton2 = Button([300,100],'E')
# myButton3 = Button([400,100],'R')
buttonList = []
for i in range(len(keys)):
    for j, key in enumerate(keys[i]):
        buttonList.append(Button([100 * j + 50, 100 * i + 50], key))

while True:
    success, img = cap.read()
    img= detector.findHands(img)
    lmlist, bboxInfo = detector.findPosition(img)
    drawALL(img, buttonList)

    if lmlist:
        for button in buttonList:
            x,y = button.pos
            w,h = button.size
            if x<lmlist[8][0]<x+w and y<lmlist[8][1] < y+h:
                cv2.rectangle(img, button.pos, (x + w, y + h), (255, 0, 0), cv2.FILLED)
                cv2.putText(img, button.text, (x + 20, y + 65), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
                l,_,_=detector.findDistance(8,12,img)
                # print(l)
                if l < 50:
                    keyboard.press(button.text)
                    cv2.rectangle(img, button.pos, (x + w, y + h), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, button.text, (x + 20, y + 65), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
                    ClickedText += button.text
                    sleep(0.2)
    cv2.rectangle(img, (55,345), (700,450), (255, 0, 0), cv2.FILLED)
    cv2.putText(img, ClickedText, (60,425), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

    # cv2.rectangle(img,(100,100),(200,200),(0,0,0),cv2.FILLED)
    # cv2.putText(img,'Q',(120,180),cv2.FONT_HERSHEY_SIMPLEX,3,(255,255,255),5)
    # myButton = Button([100,100],'Q')

    # img = myButton1.draw(img)
    # img = myButton2.draw(img)
    # img = myButton3.draw(img)
    cv2.imshow('camera',img)
    cv2.waitKey(1)